# Google Analytics- Customer Revenue Retention

by [Raul Maldonado](https://www.linkedin.com/in/raulm8/)

![GA Image](https://www.digitaltechnology.institute/wp-content/uploads/2018/03/google-analytics.gif)



## Introduction

RStudio, the developer of free and open tools for R and enterprise-ready products for teams to scale and share work, has [partnered with Google Cloud and Kaggle](https://www.kaggle.com/c/ga-customer-revenue-prediction) to demonstrate the business impact that thorough data analysis can have.

We analyze Google Merchandise Store (GStore) customer dataset to for associations in revenue and customers, and how to predict for customer revenue.

## Import

In [160]:
import os

import warnings
warnings.filterwarnings("ignore")

import pandas as pd

In [161]:
parse_dates = ['date']
ga_trainDf = pd.read_csv('../Resources/Data/PreparedData/ga_analytics_filtered_dataset.csv',\
                         parse_dates=parse_dates)

In [162]:
# ga_trainDf[ga_trainDf.fullVisitorId ==1957458976293878100]
ga_trainDf[ga_trainDf.estimatedCartTotal > 0]

,channelGrouping,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,Continent,Sub-Continent,estimatedCartTotal,visits,pageviews,bounces,newVisits,sessionQualityDim,deviceType,Region
0,Organic Search,2017-10-16,1259490915281096752,2,Not Socially Engaged,1508165159,2,1508165159,Europe,Western Europe,4208.8,1,2,32,1,0,Safari,EMEA
1,Organic Search,2017-10-16,6733325364864317729,2,Not Socially Engaged,1508160304,1,1508160304,Asia,Southeast Asia,5928.5,1,2,62,1,0,Chrome,NaN
3,Organic Search,2017-10-16,6097905367189223328,3,Not Socially Engaged,1508218860,1,1508218860,Americas,Northern America,3689.2,1,3,22,1,0,Chrome,North America
4,Display,2017-10-16,9015797273080767698,3,Not Socially Engaged,1508206108,1,1508206108,Americas,Northern America,2228.9,1,3,52,1,0,Chrome,North America
5,Referral,2017-10-16,4773414387615233459,3,Not Socially Engaged,1508177180,2,1508177180,Americas,Northern America,107.8,1,3,31,2,0,Chrome,North America
6,Organic Search,2017-10-16,1409343374628859404,3,Not Socially Engaged,1508178255,1,1508178255,Europe,Eastern Europe,6976.6,1,3,54,1,0,Chrome,NaN
7,Organic Search,2017-10-16,8522673907113279488,3,Not Socially Engaged,1508139878,1,1508139878,Asia,Southern Asia,3178.2,1,3,124,1,0,UC Browser,APAC
8,Organic Search,2017-10-16,5173848483962937435,3,Not Socially Engaged,1508149889,1,1508149889,Americas,Northern America,2858.6,1,2,13,1,0,Chrome,North America
9,Organic Search,2017-10-16,8535062014738892495,4,Not Socially Engaged,1508196114,1,1508196114,Americas,Northern America,2608.8,1,4,59,1,0,Chrome,North America
10,Referral,2017-10-16,8515247298799375456,4,Not Socially Engaged,1508179931,1,1508179931,Americas,Northern America,5987.5,1,4,106,1,0,Firefox,North America


In [149]:
ga_trainDf_raw = pd.read_csv('../Resources/Data/ZipFiles/train_v2.csv.zip',\
                            compression='zip',nrows=2000, parse_dates=parse_dates)

In [158]:
ga_trainDf_raw.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552


In [159]:
def hit_productPrice_agg(raw_hit_string):
    
    #Modify String to remove ', True", and d's items
    modifiedString = raw_hit_string.replace("'", '"').replace(\
                            "True","\"True\"").replace(\
                            "d\"s","ds")
    
    # Partition Modified string into array by ',' delim
    modifiedStringArray = modifiedString.split(",")

#    # Obtain all Product Name occurences from hit record
#     productNames = [i.replace("\"","") for i in modifiedStringArray if i.find("v2ProductName") != -1]
    
    # Obtain all Pricing occurences from hit record, respective 
    # from the above productNames list.
    # Note: Exclude first blank value in string
    productPrices = [i.replace("\"","")[1:] for i in modifiedStringArray if i.find("productPrice") != -1]

    # Obtain the value/100000 from key-pair value in each row
    productSum_estimated = [int(i.split(":")[1])/100000 for i in productPrices]

    # Sum expected total Order Value from Cart.
    estimatedAggTotal = sum(productSum_estimated)
    return(estimatedAggTotal)
ga_trainDf_raw.hits.apply(hit_productPrice_agg)

0         569.6
1        2668.5
2        1359.3
3        2228.9
4           0.0
5         299.8
6         134.8
7           0.0
8         134.8
9        4208.8
10       4208.8
11       4468.6
12        849.3
13       1828.3
14       1479.4
15       2363.7
16      12917.7
17       3428.3
18       4399.3
19       5928.5
20       2363.7
21       2228.9
22       3207.9
23       2942.8
24      12097.7
25       4747.7
26      13787.7
27      14152.8
28       9738.8
29          0.0
         ...   
1970        0.0
1971        0.0
1972        0.0
1973        0.0
1974        0.0
1975        0.0
1976        0.0
1977        0.0
1978     4399.3
1979      849.3
1980      849.3
1981     1359.3
1982     1359.3
1983     1359.3
1984     2228.9
1985     2228.9
1986     2228.9
1987     2228.9
1988     2228.9
1989     2228.9
1990     2228.9
1991     2228.9
1992     2228.9
1993     2228.9
1994     2228.9
1995     2228.9
1996     2228.9
1997     2228.9
1998     2228.9
1999     2228.9
Name: hits, Length: 2000

In [110]:
def load_df(csv_path='../Resources/Data/ZipFiles/train_v2.csv.zip', nrows=10):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df
load_df()['totals.totals'][0]

Loaded train_v2.csv.zip. Shape: (10, 19)


{'bounces': '1',
 'hits': '1',
 'newVisits': '1',
 'pageviews': '1',
 'sessionQualityDim': '1',
 'visits': '1'}

## Hypotheses and Ideas

**Objective**:

Find opportunities/signals for predicting customer revenue from Google Merchandising Store


**Hypotheses**

Expected Revenue Indicators:

* CTR

* Conversion Rate

* Visit Number (Higher the better probability of customer spending on item?)

* Total Cart Size

* Total order value size

* Average order value size (from all customers?)

**Ideas**:

In the end, I would like to have an aggregated dataset to feed into a model with the potential following signals:

1. Attributes of customer

2. Source location from customer

3. Total Revenue spent

4. Types of products choosen

Output: FullVisitorID and PredictedRevenue

**Discussion**:

A customer logs into the store. They are gandering around, checking out all of the k0oL products Google has to offer. General interest may peak for particular items. However, they may abandon the site in this first session.


Session before purchase, they either use existing basket to make a purchase within their cart.

In [5]:
ga_trainDf.head(1)

,channelGrouping,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,Continent,Sub-Continent,visits,pageviews,bounces,newVisits,sessionQualityDim,deviceType,Region
0,Organic Search,2017-10-16,1259490915281096752,2,Not Socially Engaged,1508165159,2,1508165159,Europe,Western Europe,1,2,32,1,0,Safari,EMEA


In [ ]:
# Develop questions

## Define success metrics

# EDA


## Find signals for increased revenue
#### what type of platform is this?
#### Segmentation?


## Develop ML model?

